In [ ]:
from PIL import Image
from keras.applications import ResNet50
from keras.layers import GlobalAveragePooling2D, Dense
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
import numpy as np
import cv2
import os
import imgaug.augmenters as iaa
from keras.regularizers import l2
from keras.layers import Dropout
from keras.callbacks import EarlyStopping

# Paths to crack and non-crack image folders
crack_path = 'C:\\New folder\\Dr. Surya\\Project\\crack'
noncrack_path = 'C:\\New folder\\Dr. Surya\\Project\\noncrack'

# Define target size
TARGET_SIZE = (224, 224) 

# Load all image files and annotations  
crack_images = [(os.path.join(crack_path, f), 1) for f in os.listdir(crack_path)] 
noncrack_images = [(os.path.join(noncrack_path, f), 0) for f in os.listdir(noncrack_path)]

images = crack_images + noncrack_images
np.random.shuffle(images)

# Preprocess and extract features  
def preprocess(image, label):  
    img = Image.open(image)
    img = img.resize(TARGET_SIZE)  
    x = np.array(img)/255

    return x, label

X = []
y = []

seq = iaa.Sequential([
  iaa.Fliplr(0.5), # horizontal flips
  iaa.Affine(rotate=(-10, 10)), # roations
  iaa.Affine(scale=(0.8, 1.2)), # zooms
  iaa.Affine(translate_percent=(-0.1, 0.1)) # shifts
], 
                    random_order=True)  

for img, label in images:
    x, label = preprocess(img, label)
  
    X.append(x)
    y.append(label)
    
    # Apply augmentation
    augmented_img = seq.augment_image(x)
    augmented_label = label

    X.append(augmented_img)
    y.append(augmented_label)



# Split train/test  
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Resize images to a consistent size
resized_images_train = []
for img in X_train:
    resized_img = cv2.resize(img, (224, 224))
    resized_img = cv2.convertScaleAbs(resized_img) # Convert to CV_8U format
    resized_img = cv2.cvtColor(resized_img, cv2.COLOR_BGR2RGB) # Convert to RGB format
    resized_images_train.append(resized_img)

resized_images_test= []
for img in X_test:
    resized_img = cv2.resize(img, (224, 224))
    resized_img= cv2.convertScaleAbs(resized_img) # Convert to CV_8U format
    resized_img= cv2.cvtColor(resized_img, cv2.COLOR_BGR2RGB) # Convert to RGB format
    resized_images_test.append(resized_img)

# Convert resized_images to NumPy array
X_train = np.empty((len(resized_images_train), 224, 224, 3), dtype=np.float32)
for i, img in enumerate(resized_images_train):
    X_train[i] = img

X_test = np.empty((len(resized_images_test), 224, 224, 3), dtype=np.float32)
for i, img in enumerate(resized_images_test):
    X_test[i] = img


# Convert to NumPy arrays
y_train = np.array(y_train) 
y_test = np.array(y_test)

# Create model
base_model = ResNet50(weights='imagenet', include_top=False, 
                input_shape=(224,224,3))
x = base_model.output  
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
x = Dense(1024, activation='relu',kernel_regularizer=l2(0.01))(x)  # Add regularizer here
preds = Dense(1, activation='sigmoid')(x)  # Use sigmoid activation for binary classification)(x)


model = Model(base_model.input, preds)


# Train model
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=1e-4), metrics=['accuracy'])
checkpoint = ModelCheckpoint('C:\\New folder\\Dr. Surya\\Project\\CNN2.h5', save_best_only=True)
early_stop = EarlyStopping(monitor='val_loss', patience=3)

model.fit(X_train, y_train, epochs=20, validation_data=(X_test,y_test), 
          callbacks=[checkpoint, early_stop])

# Load best model and evaluate
model.load_weights('C:\\New folder\\Dr. Surya\\Project\\CNN2.h5')
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {accuracy}')